In [1]:
#from run_ib import RunIB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stirling import stirling
import enumerate_lexicons
pd.options.display.max_rows = 250
import scipy
from helper_functions import *
from run_ib_new import *


In [2]:
# helper function: _ib (this includes custom initialization)
DEFAULT_NUM_ITER=10
PRECISION = 1e-16
def _ib(p_x, p_y_x, Z, gamma, init, num_iter=DEFAULT_NUM_ITER, temperature = 1):
    """ Find encoder q(Z|X) to minimize J = I[X:Z] - gamma * I[Y:Z].
    
    Input:
    p_x : Distribution on X, of shape X.
    p_y_x : Conditional distribution on Y given X, of shape X x Y.
    gamma : A non-negative scalar value.
    Z : Support size of Z.

    Output: 
    Conditional distribution on Z given X, of shape X x Z.

    """
    # Support size of X
    X = p_x.shape[-1]

    # Support size of Y
    Y = p_y_x.shape[-1]

    # Randomly initialize the conditional distribution q(z|x)
    q_z_x = init #scipy.special.softmax(np.random.randn(X, Z), -1) # shape X x Z
    p_y_x = p_y_x[:, None, :] # shape X x 1 x Y
    p_x = p_x[:, None] # shape X x 1

    # Blahut-Arimoto iteration to find the minimizing q(z|x)
    for _ in range(num_iter):
        q_xz = p_x * q_z_x # Joint distribution q(x,z), shape X x Z
        q_z = q_xz.sum(axis=0, keepdims=True) # Marginal distribution q(z), shape 1 x Z
        q_y_z = ((q_xz / q_z)[:, :, None] * p_y_x).sum(axis=0, keepdims=True) # Conditional decoder distribution q(y|z), shape 1 x Z x Y
        d = ( 
            scipy.special.xlogy(p_y_x, p_y_x)
            - scipy.special.xlogy(p_y_x, q_y_z) # negative KL divergence -D[p(y|x) || q(y|z)]
        ).sum(axis=-1) # expected distortion over Y; shape X x Z
        q_z_x = scipy.special.softmax((np.log(q_z) - gamma*d)/temperature, axis=-1) # Conditional encoder distribution q(z|x) = 1/Z q(z) e^{-gamma*d}

    return q_z_x

num_dists = 3
pgs_dists = [0,0.789,-1.315]
num_words = 9
num_meanings = num_dists * 3

# function to make the non-deterministic frontier
def make_curve(mu, logsp=np.logspace(2, 0, num=1500), pgs=pgs_dists):
    init = np.identity(num_words)

    qW_M = []
    informativity = []
    complexity = []

    for gamma in logsp:
        # x = RunIB(mu, gamma, num_dists, pgs_dists)
        # shape: M x U x W
        x = RunIB(mu, num_dists, pgs_dists)
        p_m = x.prior
        p_u_m = x.prob_u_given_m
        p_um = p_m[:, None] * p_u_m
        p_u = p_um.sum(axis=-1, keepdims=True)
        q_w_m = _ib(p_m, p_u_m, num_words, gamma, init, num_iter = 20)
        informativity_temp, complexity_temp = information_plane(p_m, p_u_m, q_w_m)

        qW_M.append(q_w_m)
        informativity.append(informativity_temp)
        complexity.append(complexity_temp)
        init = q_w_m
        
    curve = pd.DataFrame(data = {'gamma': logsp,
                    'informativity' : informativity,
                    'complexity' : complexity,
                    'J' : complexity - logsp*informativity})
    return curve, qW_M

# function to find the objective function
def get_objective(p_m, p_u_m, q_w_m, gamma):
    informativity, complexity = information_plane(p_m, p_u_m, q_w_m)
    return complexity - gamma * informativity

# function to find gamma minimizing the objective function
def find_gamma_index(p_m, p_u_m, q_w_m, curve):
    objs = np.array([get_objective(p_m, p_u_m, q_w_m, gamma) for gamma in logsp])
    objs_2 = curve["J"].values
    diff = objs - objs_2
    return(diff.argmin())

# function to calculate efficiency loss (Zaslavsky et al., 2018)
def find_epsilon(p_m, p_u_m, q_w_m, curve):
    objs = np.array([get_objective(p_m, p_u_m, q_w_m, gamma) for gamma in logsp])
    objs_2 = curve["J"].values
    diff = objs - objs_2
    return(diff.min()/logsp[diff.argmin()])

# codes from Zaslavsky et al. (2018)
def xlogx(v):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(v > PRECISION, v * np.log2(v), 0)
    
def H(p, axis=None):
    """ Entropy """
    return -xlogx(p).sum(axis=axis)

def MI(pXY):
    """ mutual information, I(X;Y) """
    return H(pXY.sum(axis=0)) + H(pXY.sum(axis=1)) - H(pXY)

# function to calculate gNID (Zaslavsky et al., 2018)
def gNID(pW_X, pV_X, pX):
    if len(pX.shape) == 1:
        pX = pX[:, None]
    elif pX.shape[0] == 1 and pX.shape[1] > 1:
        pX = pX.T
    pXW = pW_X * pX
    pWV = pXW.T.dot(pV_X)
    pWW = pXW.T.dot(pW_X)
    pVV = (pV_X * pX).T.dot(pV_X)
    score = 1 - MI(pWV) / (np.max([MI(pWW), MI(pVV)]))
    return score

In [3]:
# This part generates the non-deterministic optimal frontier, with gamma ranging from 1 to 1000 (it takes ~2h)
logsp = np.logspace(3,0,num = 1500)
mu = 0.2
curve, qW_M = make_curve(mu, logsp, pgs_dists)

In [4]:
# This part generates all possible lexicons
x = RunIB(mu, num_dists, pgs_dists)
num_meanings = 3 * num_dists
lexicon_size_range = range(2, num_meanings + 1)
sim_lex_dict = {lexicon_size: [lexicon for lexicon in enumerate_lexicons.enumerate_possible_lexicons(num_meanings, lexicon_size)] for 
        lexicon_size in lexicon_size_range}

In [5]:
# Data frame containing real lexicons
lexicons = x.get_real_langs(num_meanings)

df = pd.DataFrame([{dm: l[1].argmax(axis=1)[dm_num]
                        for dm_num, dm in enumerate(x.deictic_index)} for l in lexicons])
information_plane_list = [information_plane(x.prior, x.prob_u_given_m, l[1]) for l in lexicons]
df["I[U;W]"] = [l[0] for l in information_plane_list]
df["I[M;W]"] = [l[1] for l in information_plane_list]
df["gamma_fit"] = [logsp[find_gamma_index(x.prior, x.prob_u_given_m, l[1], curve)] for l in lexicons]
df["epsilon"] = [find_epsilon(x.prior, x.prob_u_given_m, l[1], curve) for l in lexicons]
df["gNID"] = [gNID(l[1], qW_M[find_gamma_index(x.prior, x.prob_u_given_m, l[1], curve)], x.prior) for l in lexicons]
df["Language"] = [l[0] for l in lexicons]
df["Area"] = [l[2] for l in lexicons]

/Users/cshnican/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/cshnican/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/cshnican/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/cshnican/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
# Data frame containing simulated lexicons
lexicons_sim = []
for lexicon_size in range(2, num_meanings+1):
    all_lex = sim_lex_dict[lexicon_size]
    lexicons_sim += [("simulated", l[1], "simulated") for l in all_lex]

df_sim = pd.DataFrame([{dm: l[1].argmax(axis=1)[dm_num]
                        for dm_num, dm in enumerate(x.deictic_index)} for l in lexicons_sim])

information_plane_list_sim = [information_plane(x.prior, x.prob_u_given_m, l[1]) for l in lexicons_sim]
df_sim["I[U;W]"] = [l[0] for l in information_plane_list_sim]
df_sim["I[M;W]"] = [l[1] for l in information_plane_list_sim]
df_sim["gamma_fit"] = [logsp[find_gamma_index(x.prior, x.prob_u_given_m, l[1], curve)] for l in lexicons_sim]
df_sim["epsilon"] = [find_epsilon(x.prior, x.prob_u_given_m, l[1], curve) for l in lexicons_sim]
df_sim["gNID"] = [gNID(l[1], qW_M[find_gamma_index(x.prior, x.prob_u_given_m, l[1], curve)], x.prior) for l in lexicons_sim]
df_sim["Language"] = [l[0] for l in lexicons_sim]
df_sim["Area"] = [l[2] for l in lexicons_sim]

In [7]:
df = consistency(df)
df_sim = consistency(df_sim)

i =  1  /  220 ; lang =  	Kabiyé (Atlantic-Congo, Gur)


/Users/cshnican/Documents/BCS/TedLab/here_there_way_over_there/repo_published/spatial_demonstratives/helper_functions.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Language'][i] = df['Language'][i] + str(k)


i =  1  /  21146 ; lang =  simulated152
i =  501  /  21146 ; lang =  simulated3002
i =  1001  /  21146 ; lang =  simulated1637
i =  1501  /  21146 ; lang =  simulated565
i =  2001  /  21146 ; lang =  simulated1792
i =  2501  /  21146 ; lang =  simulated266
i =  3001  /  21146 ; lang =  simulated809
i =  3501  /  21146 ; lang =  simulated14302
i =  4001  /  21146 ; lang =  simulated6929
i =  4501  /  21146 ; lang =  simulated1562
i =  5001  /  21146 ; lang =  simulated6070
i =  5501  /  21146 ; lang =  simulated5189
i =  6001  /  21146 ; lang =  simulated6499
i =  6501  /  21146 ; lang =  simulated6160
i =  7001  /  21146 ; lang =  simulated12277
i =  7501  /  21146 ; lang =  simulated1832
i =  8001  /  21146 ; lang =  simulated14741
i =  8501  /  21146 ; lang =  simulated7838
i =  9001  /  21146 ; lang =  simulated12560
i =  9501  /  21146 ; lang =  simulated13475
i =  10001  /  21146 ; lang =  simulated7791
i =  10501  /  21146 ; lang =  simulated10573
i =  11001  /  21146 ; lang =  s

In [8]:
# Save things
df.to_csv('sheets/real_lexicons_fit_mu_' + str(mu) + '_pgs_' + "_".join([str(pgs) for pgs in pgs_dists]) + 'num_dists_' + str(num_dists) + '.csv')
df_sim.to_csv('sheets/sim_lexicons_fit_mu_'+ str(mu) + '_pgs_' + "_".join([str(pgs) for pgs in pgs_dists]) + 'num_dists_' + str(num_dists) +'.csv')
curve.to_csv('sheets/ib_curve_non_deter_mu_' + str(mu) + '_pgs_' + "_".join([str(pgs) for pgs in pgs_dists]) + 'num_dists_' + str(num_dists) +'.csv')